In [11]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Processed CyberBullying file.csv')
print(df.shape)

#remove null values

df.dropna(inplace=True)
df.isnull().sum()
print(df.columns)

(32450, 3)
Index(['text', 'tags', 'processed'], dtype='object')


In [4]:
df.head(10)

,text,tags,processed
0,so what? \n\nwhat should i do? then promise to...,0,? ? promise never edit wikipedia get unblocked...
1,"""\n\nAnd PBS as you are a bit of an expert on ...",0,`` pbs bit expert wiki-policies know read some...
2,WTF! \n\nu fckin faggot ass mutha fcker u jus ...,1,wtf ! u fckin faggot ass mutha fcker u jus re-...
3,"Great, I'm glad we managed to work this out.",0,"great , 'm glad managed work ."
4,"""\n\n Hu12 administrator apparent libel and ab...",0,`` hu12 administrator apparent libel abuse rep...
5,Hi! I am back again!\nLast warning!\nStop undo...,1,hi ! back ! last warning ! stop undoing edits ...
6,you stink \nYou suck HangingCurve/Blueboy96. E...,1,stink suck hangingcurve/blueboy96 . even admin...
7,crash rules \n\nThe Rules of Wedding Crashing ...,0,crash rules rules wedding crashing created del...
8,"""\nDoes this look good? I changed it as the so...",0,`` look good ? changed source talk theme music...
9,Wikipedia:Requests for arbitration/Husnock\nHe...,0,wikipedia : requests arbitration/husnock hello...


In [5]:
#check GPU Availability
import tensorflow as tf
tf.test.is_gpu_available()

False

In [6]:
df.describe()

# classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
classes = ['Toxic', 'Not Toxic']
# targets = df[classes].values
targets = df['tags'].values

#perform feature engineering using GloVe Embeddings

embedding_file_path = 'glove.6B.100d.txt' #GloVe embedding file. Almost 350MB

In [7]:
targets[:12]

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1])

In [9]:
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Activation, Dropout, CuDNNGRU
from tensorflow.keras.layers import RNN, concatenate
from tensorflow.keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPool1D

from tensorflow.keras import initializers, optimizers

from tensorflow.keras.callbacks import ModelCheckpoint

In [10]:
features = df['text'].fillna(' ').str.lower()

In [11]:
max_features, embed_size, maxlen = 100000, 100, 150
tokenizer = Tokenizer(max_features)

tokenizer.fit_on_texts(list(features))
sentences = tokenizer.texts_to_sequences(features)

In [12]:
sentences[0]

[34, 35, 35, 60, 5, 27, 87, 2928, 3, 188, 88, 17, 28, 6, 5, 41, 71, 2007]

In [13]:
padded_features = pad_sequences(sentences, maxlen=maxlen)

In [14]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

#costly operations, do it at your own risk
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embedding_file_path))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))+1
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
max_features = nb_words

In [40]:
#build the Deep Learning model

orig_input = Input(shape=(maxlen,)) #series
embed_layer = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable=False)(orig_input)
bi_layer = Bidirectional(GRU(128, return_sequences=True))(embed_layer)
avg_pool = GlobalAveragePooling1D()(bi_layer)
max_pool = GlobalMaxPool1D()(bi_layer)

conc_layer = concatenate([avg_pool, max_pool])
prediction_layer = Dense(1, activation='sigmoid')(conc_layer)

model = Model(inputs=orig_input, outputs=prediction_layer)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(clipnorm=1.))

print(model.summary())


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 100)     7029300     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 150, 256)     175872      embedding_2[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_2 (Glo (None, 256)          0           bidirectional_2[0][0]            
____________________________________________________________________________________________

In [19]:
saved_model_file = 'best_acc_meer_model_weights.hdf5'

ckpt = ModelCheckpoint(saved_model_file, monitor='val_acc', save_best_only=True, save_weights_only=True, mode='max')



In [20]:
bs, ep = 32, 10

model.fit(padded_features, targets, validation_split=0.3, epochs=ep, batch_size=bs, callbacks=[ckpt])

Train on 22715 samples, validate on 9735 samples
Epoch 1/10
22715/22715 [==============================] - 194s 9ms/sample - loss: 0.2885 - acc: 0.8753 - val_loss: 0.2405 - val_acc: 0.9004
Epoch 2/10
22715/22715 [==============================] - 216s 10ms/sample - loss: 0.2225 - acc: 0.9087 - val_loss: 0.2184 - val_acc: 0.9084
Epoch 3/10
22715/22715 [==============================] - 185s 8ms/sample - loss: 0.1938 - acc: 0.9212 - val_loss: 0.2091 - val_acc: 0.9138
Epoch 4/10
22715/22715 [==============================] - 186s 8ms/sample - loss: 0.1631 - acc: 0.9353 - val_loss: 0.2124 - val_acc: 0.9118
Epoch 5/10
22715/22715 [==============================] - 187s 8ms/sample - loss: 0.1332 - acc: 0.9482 - val_loss: 0.2109 - val_acc: 0.9149
Epoch 6/10
22715/22715 [==============================] - 185s 8ms/sample - loss: 0.1029 - acc: 0.9626 - val_loss: 0.2294 - val_acc: 0.9119
Epoch 7/10
22715/22715 [==============================] - 179s 8ms/sample - loss: 0.0759 - acc: 0.9747 - val_l

In [45]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [41]:
import pickle

with open('tokenizer.pickle','wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

#save tokenizer to fit on texts during testing.

In [42]:
#save embedding matrix, model
with open('embedding_matrix.pickle', 'wb') as f:
    pickle.dump(embedding_matrix,f,protocol=pickle.HIGHEST_PROTOCOL)

model.save('Toxic_Comments_Model.model')

In [3]:
#call a testing / prediction line.

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

def test_prediction(sent, tokenizer='tokenizer.pickle', model='Toxic_Comments_Model.model'):

    
    if len(sent) == 0:
        print("No input entered - please enter a valid input")
        return
        
    import numpy as np
    import nltk
    import os

    import pickle

    from tensorflow.keras.models import Model
    #from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.models import load_model
    from tensorflow.keras.models import model_from_json
    from tensorflow.keras import optimizers

    from tensorflow.keras.preprocessing.sequence import pad_sequences

    #do all the preprocessing you did earlier

    max_features, embed_size, maxlen = 100000, 100, 150

    tokenizer_file = open(tokenizer,'rb')
    tokenizer = pickle.load(tokenizer_file)
    tokenizer_file.close()

    features = sent.lower()

    sentences = tokenizer.texts_to_sequences([features])

    padded_features = pad_sequences(sentences, maxlen=maxlen) #features

    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("best_acc_meer_model_weights.hdf5")
    loaded_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(clipnorm=1.))
    print("Loaded model from disk")

    #model = load_model('Toxic_Comments_Model.model') #load the trained model
    #model.predict()

    label_list = loaded_model.predict(padded_features)
    print(label_list)

    # predict_labels = []
    # for item in label_list[0]:
        # predict_labels.append(int(round(item)))
    
    # classes = ["Toxic",", Severely Toxic",", Obscene",", a Threat",", an Insult",", Identity Hate"]
    classes = ['Not Toxic', 'Toxic']

    print("The sentence : '{}', is classified as : \n".format(sent))
    returned_class = classes[1] if label_list[0][0]>=0.5 else classes[0]
    # print(returned_class)

    """
    for i in range(len(classes)):
        if predict_labels[i] == 1:
            print(classes[i],end=" ")
    """
    proba = 1-label_list[0][0] if label_list[0][0] < 0.5 else label_list[0][0] 
    return proba, returned_class
    

In [4]:
proba, label = test_prediction('I am hungry, you bitch. My hatred for you knows no bounds')
print('PROBABILITY : {} | LABEL : {}'.format(proba, label))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loaded model from disk
[[0.9911705]]
The sentence : 'I am hungry, you bitch. My hatred for you knows no bounds', is classified as : 

PROBABILITY : 0.9911705255508423 | LABEL : Toxic


In [ ]:
!pip list

Package                  Version        
------------------------ ---------------
absl-py                  0.9.0          
alabaster                0.7.12         
albumentations           0.1.12         
altair                   4.1.0          
asgiref                  3.2.7          
astor                    0.8.1          
astropy                  4.0.1.post1    
astunparse               1.6.3          
atari-py                 0.2.6          
atomicwrites             1.3.0          
attrs                    19.3.0         
audioread                2.1.8          
autograd                 1.3            
Babel                    2.8.0          
backcall                 0.1.0          
backports.tempfile       1.0            
backports.weakref        1.0.post1      
beautifulsoup4           4.6.3          
bleach                   3.1.4          
blis                     0.4.1          
bokeh                    1.4.0          
boto                     2.49.0         
boto3           

In [36]:
!pip freeze > requirements.txt